In [2]:
!pip install pandas


In [3]:
import pandas as pd
import glob
import os

## Attach the end location home information

In [5]:
# Define the file path
file_path = '/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Souradeep_Trip_Info/RWRAD_all_label.csv'

# Read the CSV file into a DataFrame
labels_df = pd.read_csv(file_path)

In [11]:
labels_df.head()

,subj,time_utc,labels
0,RWRAD_001,2021-03-18 18:21:48,home
1,RWRAD_001,2021-03-18 21:23:42,home
2,RWRAD_001,2021-03-18 23:58:10,home
3,RWRAD_001,2021-03-19 14:30:10,home
4,RWRAD_001,2021-03-19 15:30:53,home


In [9]:
# Drop column 'B'
labels_df = labels_df.drop(columns=["Unnamed: 0"])

In [17]:
import pandas as pd
import os

# Define the path to the folder containing GPS files
folder_path = r"C:\Users\aparnaj8\Box\Data (Matthew Rizzo)\DataAnalysis\ReferenceDatabases\BlackBox_GPSDrivingLocations\BlackBox_GPSDrivingLocations_1hz"

# Initialize a list to store the final results and processed files
results = []
processed_files = []  # List to track processed files

# Iterate over each unique subject in labels_df
for subj in labels_df['subj'].unique():
    # Construct the file path for the subject's GPS file
    file_name = f"{subj}_BlackBox_GPSDrivingLocations_1hz.csv"
    file_path = os.path.join(folder_path, file_name)

    # Check if the file exists
    if os.path.exists(file_path):
        print(f"Started processing file: {file_name}")  # Announce processing start

        # Load the GPS file into a DataFrame
        gps_df = pd.read_csv(file_path)

        # Ensure time columns are in datetime format for proper comparison
        labels_df['time_utc'] = pd.to_datetime(labels_df['time_utc'])
        gps_df['time_utc'] = pd.to_datetime(gps_df['time_utc'])

        # Filter gps_df for rows where time_cat = 'end'
        gps_df_filtered = gps_df[gps_df['time_cat'] == 'end']

        # Filter the labels_df for the current subject
        subj_labels = labels_df[labels_df['subj'] == subj]

        # Perform a left merge to retain all rows from labels_df
        merged_df = pd.merge(
            subj_labels,
            gps_df_filtered[['time_utc', 'drive', 'gps_lat', 'gps_long']],  # Columns to match and attach
            on='time_utc',
            how='left'  # Use 'left' to keep all rows from labels_df
        )

        # Append the merged data to the results list
        results.append(merged_df)

        # Mark the file as processed
        processed_files.append(file_name)
    else:
        print(f"File not found: {file_path}")

# Combine all results into a single DataFrame
if results:
    final_df = pd.concat(results, ignore_index=True)
else:
    final_df = pd.DataFrame()  # Handle case where no data is processed

# Display the final DataFrame
print(final_df)

# Display the list of processed files
print("\nFiles Processed:")
for file in processed_files:
    print(f" - {file}")




Started processing file: RWRAD_001_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_002_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_003_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_004_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_005_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_006_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_007_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_008_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_009_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_010_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_011_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_013_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_015_BlackBox_GPSDrivingLocations_1hz.csv
Started processing file: RWRAD_017_BlackBox_GPSDrivingLocations_

In [19]:
final_df.head()

,subj,time_utc,labels,drive,gps_lat,gps_long
0,RWRAD_001,2021-03-18 18:21:48,home,4,40.779259,-96.607689
1,RWRAD_001,2021-03-18 21:23:42,home,5,40.779266,-96.607674
2,RWRAD_001,2021-03-18 23:58:10,home,7,40.779259,-96.607796
3,RWRAD_001,2021-03-19 14:30:10,home,9,40.779236,-96.607628
4,RWRAD_001,2021-03-19 15:30:53,home,12,40.779263,-96.607636


In [23]:
len(labels_df)

34843

In [27]:
# Count the total number of NaN values in each column
nan_counts_per_column = final_df.isna().sum()

# Print the count of NaN values for each column
print("Count of NaN values per column:")
print(nan_counts_per_column)

# Count the total number of rows with any NaN values
rows_with_nan = final_df.isna().any(axis=1).sum()
print(f"\nTotal number of rows with NaN values: {rows_with_nan}")


Count of NaN values per column:
subj        0
time_utc    0
labels      0
drive       0
gps_lat     0
gps_long    0
dtype: int64

Total number of rows with NaN values: 0


In [25]:
# Save the final DataFrame to a CSV file (if needed)
output_path = r"/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Trip_variable/RWRAD_end_trip_info.csv"
final_df.to_csv(output_path, index=False)

In [29]:
final_df.head()

,subj,time_utc,labels,drive,gps_lat,gps_long
0,RWRAD_001,2021-03-18 18:21:48,home,4,40.779259,-96.607689
1,RWRAD_001,2021-03-18 21:23:42,home,5,40.779266,-96.607674
2,RWRAD_001,2021-03-18 23:58:10,home,7,40.779259,-96.607796
3,RWRAD_001,2021-03-19 14:30:10,home,9,40.779236,-96.607628
4,RWRAD_001,2021-03-19 15:30:53,home,12,40.779263,-96.607636


In [38]:
final_df['subj'].nunique()

153

##  Extract only home location

In [17]:
# Define the file path
file_path = r"/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Trip_variable/RWRAD_end_trip_info.csv"

# Read the CSV file into a DataFrame
final_df= pd.read_csv(file_path)

In [19]:
final_df.head()

,subj,time_utc,labels,drive,gps_lat,gps_long
0,RWRAD_001,2021-03-18 18:21:48,home,4,40.779259,-96.607689
1,RWRAD_001,2021-03-18 21:23:42,home,5,40.779266,-96.607674
2,RWRAD_001,2021-03-18 23:58:10,home,7,40.779259,-96.607796
3,RWRAD_001,2021-03-19 14:30:10,home,9,40.779236,-96.607628
4,RWRAD_001,2021-03-19 15:30:53,home,12,40.779263,-96.607636


In [21]:
# Filter the DataFrame for rows where the 'labels' column is 'home'
home_df = final_df[final_df['labels'] == 'home']
home_df.head()


,subj,time_utc,labels,drive,gps_lat,gps_long
0,RWRAD_001,2021-03-18 18:21:48,home,4,40.779259,-96.607689
1,RWRAD_001,2021-03-18 21:23:42,home,5,40.779266,-96.607674
2,RWRAD_001,2021-03-18 23:58:10,home,7,40.779259,-96.607796
3,RWRAD_001,2021-03-19 14:30:10,home,9,40.779236,-96.607628
4,RWRAD_001,2021-03-19 15:30:53,home,12,40.779263,-96.607636


In [23]:
len(home_df)

11095

In [25]:
home_df['subj'].nunique()

136

## Extract first value of lat and long for every subject

In [27]:
# Group by 'subj' and keep the first entry for each subject based on gps_lat and gps_long
home_df_first_entry = home_df.groupby('subj').first().reset_index()

In [29]:
home_df_first_entry

,subj,time_utc,labels,drive,gps_lat,gps_long
0,RWRAD_001,2021-03-18 18:21:48,home,4,40.779259,-96.607689
1,RWRAD_002,2021-03-20 19:57:43,home,3,41.235500,-95.965805
2,RWRAD_003,2021-03-26 18:55:26,home,12,41.325111,-96.167122
3,RWRAD_004,2021-03-25 01:08:58,home,1,41.251835,-95.993309
4,RWRAD_005,2021-03-27 17:44:32,home,2,41.235455,-95.965858
...,...,...,...,...,...,...
131,RWRAD_157,2023-08-24 17:55:10,home,3,41.273113,-96.033150
132,RWRAD_158,2023-09-11 15:04:36,home,14,41.152775,-96.184402
133,RWRAD_160,2023-10-01 22:48:48,home,6,41.230373,-96.026009
134,RWRAD_161,2023-11-17 00:11:04,home,6,41.020580,-96.173141


## Calculate the distance for all Csvs ( some csv files are misisng due to lack of home info)

In [152]:
import os
import pandas as pd
from geopy.distance import geodesic

# Initialize the final output dataframe
final_output_df = pd.DataFrame(columns=['subj', 'drive', 'distance', 'label'])

# Path to the GPS data files
gps_files_path = r"C:\Users\aparnaj8\Box\Data (Matthew Rizzo)\DataAnalysis\ReferenceDatabases\BlackBox_GPSDrivingLocations\BlackBox_GPSDrivingLocations_1hz"

# Get all CSV files from the folder
gps_files = [file for file in os.listdir(gps_files_path) if file.endswith(".csv")]

# Iterate through all CSV files
for file in gps_files:
    # Load the current GPS file
    gps_df = pd.read_csv(os.path.join(gps_files_path, file))
    gps_df = gps_df[gps_df['time_cat'] == 'start']  # Filter for 'start' only

    # Print which file is being processed
    print(f"Processing file: {file}")

    # Iterate over each row in home_df_first_entry to match with GPS data
    for index, row in home_df_first_entry.iterrows():
        # Match the subject in the GPS data
        matching_gps = gps_df[gps_df['subj'] == row['subj']]

        if not matching_gps.empty:
            # Calculate the geodesic distance for each match if coordinates are valid
            home_lat = row['gps_lat']
            home_lon = row['gps_long']

            # Skip if the home coordinates are missing or invalid
            if pd.isna(home_lat) or pd.isna(home_lon):
                continue

            home_point = (home_lat, home_lon)
            
            # For each matching entry, calculate the distance and label
            for _, gps_row in matching_gps.iterrows():
                gps_lat = gps_row['gps_lat']
                gps_lon = gps_row['gps_long']
                
                # Skip if GPS coordinates are missing or invalid
                if pd.isna(gps_lat) or pd.isna(gps_lon):
                    continue

                gps_point = (gps_lat, gps_lon)

                # Calculate distance in feet
                distance = geodesic(home_point, gps_point).feet
                
                # Assign label based on distance
                label = 'home' if distance < 100 else 'not_home'
                
                # Prepare the new row as a DataFrame
                new_row = pd.DataFrame({
                    'subj': [row['subj']],
                    'drive': [gps_row['drive']],
                    'distance': [distance],
                    'label': [label]
                })
                
                # Concatenate the new row to the final dataframe
                final_output_df = pd.concat([final_output_df, new_row], ignore_index=True)


Processing file: RWRAD_001_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_002_BlackBox_GPSDrivingLocations_1hz.csv


C:\Users\aparnaj8\AppData\Local\Temp\ipykernel_21204\795760526.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_output_df = pd.concat([final_output_df, new_row], ignore_index=True)


Processing file: RWRAD_003_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_004_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_005_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_006_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_007_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_008_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_009_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_010_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_011_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_012_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_013_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_014_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_015_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_016_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_017_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_018_BlackBox_GPSD

## Check duplicate count of subj and drive

In [154]:
# Check for duplicates based on 'subj' and 'drive'
duplicates = final_output_df[final_output_df.duplicated(subset=['subj', 'drive'], keep=False)]

# Count the number of duplicates
duplicate_count = duplicates.shape[0]  # or len(duplicates)
print(f"Number of duplicate combinations of 'subj' and 'drive': {duplicate_count}")

# Optionally, you can display the duplicate rows for inspection
print(duplicates[['subj', 'drive']].head())


Number of duplicate combinations of 'subj' and 'drive': 0
Empty DataFrame
Columns: [subj, drive]
Index: []


In [163]:
# Get the count of unique 'subj' values
unique_subj_count = final_output_df['subj'].nunique()

# Print the result
print(f"Number of unique subjects: {unique_subj_count}")


Number of unique subjects: 136


## Save the dataframe as csv

In [159]:
# Optionally, save the final dataframe to a CSV
final_output_df.to_csv("RWRAD_start_trip_home_info.csv", index=False)

## Check the output file

In [142]:
len(final_output_df)

29238

In [150]:
final_output_df.head()

,subj,drive,distance,label
0,RWRAD_001,2,5867.960252,not_home
1,RWRAD_001,3,14367.450767,not_home
2,RWRAD_001,4,13534.904549,not_home
3,RWRAD_001,5,17.808028,home
4,RWRAD_001,6,11.872215,home


In [144]:
home_count = final_output_df[final_output_df['label'] == 'home'].shape[0]
print(home_count)


2507


In [148]:
# Filter the rows where the label is 'home'
home_label_df = final_output_df[final_output_df['label'] == 'home']

# Find the row with the maximum distance
max_distance_row = home_label_df.loc[home_label_df['distance'].idxmax()]

# Display the row with the maximum distance
print(max_distance_row)


subj        RWRAD_126
drive             325
distance    49.999196
label            home
Name: 22225, dtype: object


In [117]:
# Filter rows where distance is between 50 and 100 feet inclusive
filtered_df = final_output_df[(final_output_df['distance'] >= 50) & (final_output_df['distance'] <= 100)]

# Display the filtered DataFrame
print(filtered_df)


          subj drive   distance     label
9    RWRAD_001    13  83.203012  not_home
13   RWRAD_001    18  66.713582  not_home
46   RWRAD_001    59  51.762925  not_home
52   RWRAD_001    67  80.174396  not_home
60   RWRAD_001    84  57.605268  not_home
70   RWRAD_001    95  68.395875  not_home
129  RWRAD_001   168  53.626994  not_home
133  RWRAD_001   173  64.944441  not_home
155  RWRAD_001   199  56.776572  not_home
161  RWRAD_001   207  55.310644  not_home
206  RWRAD_001   263  80.930840  not_home
233  RWRAD_001   304  52.227253  not_home
235  RWRAD_001   306  54.857341  not_home
236  RWRAD_001   307  71.238585  not_home
248  RWRAD_001   327  53.983603  not_home


## Test code for one csv file

In [ ]:
import os
import pandas as pd
import numpy as np
from geopy.distance import geodesic


# Initialize the final output dataframe
final_output_df = pd.DataFrame(columns=['subj', 'drive', 'distance', 'label'])

# Path to the GPS data files
gps_files_path = r"C:\Users\aparnaj8\Box\Data (Matthew Rizzo)\DataAnalysis\ReferenceDatabases\BlackBox_GPSDrivingLocations\BlackBox_GPSDrivingLocations_1hz"

# Get the first CSV file from the folder
gps_files = [file for file in os.listdir(gps_files_path) if file.endswith(".csv")]
first_file = gps_files[0]  # The first CSV file

# Process the first file only
gps_df = pd.read_csv(os.path.join(gps_files_path, first_file))
gps_df = gps_df[gps_df['time_cat'] == 'start']  # Filter for 'start' only

# Print which file is being processed
print(f"Processing file: {first_file}")

# Iterate over each row in home_df_first_entry to match with GPS data
for index, row in home_df_first_entry.iterrows():
    # Match the subject in the GPS data
    matching_gps = gps_df[gps_df['subj'] == row['subj']]

    if not matching_gps.empty:
        # Calculate the geodesic distance for each match
        home_point = (row['gps_lat'], row['gps_long'])
        
        # For each matching entry, calculate the distance and label
        for _, gps_row in matching_gps.iterrows():
            gps_point = (gps_row['gps_lat'], gps_row['gps_long'])
            
            # Calculate distance in feet
            distance = geodesic(home_point, gps_point).feet
            
            # Assign label based on distance
            label = 'home' if distance < 50 else 'not_home'
            
            # Prepare the new row as a DataFrame
            new_row = pd.DataFrame({
                'subj': [row['subj']],
                'drive': [gps_row['drive']],
                'distance': [distance],
                'label': [label]
            })
            
            # Concatenate the new row to the final dataframe
            final_output_df = pd.concat([final_output_df, new_row], ignore_index=True)



## find the home information for the remaining subjects 

In [31]:
file_path = 'C:\\Users\\aparnaj8\\Box\\Data (Matthew Rizzo)\\DataAnalysis\\FormattingQAChecks\\REDCap_Formatting\\REDCap_Format_Data\\dlq.csv'

# Read the CSV file into a DataFrame
dlq_df = pd.read_csv(file_path)

In [32]:
dlq_df.head()

,subj,subj_type,year,visit,type,address,gps_lat,gps_long,name,preciseorapproximate
0,RWRAD_001,mci,1,1,home,8219 East Pointe Rd Lincoln| NE 68506,40.779291,-96.607658,Home,precise
1,RWRAD_001,mci,1,1,groceries,7151 Stacy Ln Lincoln| NE 68516,40.767595,-96.622539,Hy-Vee,precise
2,RWRAD_001,mci,1,1,gas,7151 Stacy Lane Lincoln| NE 68516,40.767595,-96.622539,Hy-Vee,precise
3,RWRAD_001,mci,1,1,prescriptions,7151 Stacy Lane Lincoln| NE 68516,40.767595,-96.622539,Hy-Vee,precise
4,RWRAD_001,mci,1,1,social,500 W South St #8 Lincoln| NE 68522,40.792386,-96.727334,Backswing Brewery,precise


In [33]:
# Filter the DataFrame to get rows where 'type' is 'home'
home_df = dlq_df[dlq_df['type'] == 'home']

# Get unique combinations of 'subj' and 'type'
unique_home_combinations = home_df[['subj', 'type']].drop_duplicates()

# Merge the unique combinations with the original DataFrame to get 'gps_lat' and 'gps_long'
home_gps_info = pd.merge(unique_home_combinations, dlq_df[['subj', 'type', 'gps_lat', 'gps_long']], on=['subj', 'type'], how='left')


In [37]:
home_gps_info.head()

,subj,type,gps_lat,gps_long
0,RWRAD_001,home,40.779291,-96.607658
1,RWRAD_002,home,41.235652,-95.965888
2,RWRAD_003,home,41.325306,-96.166992
3,RWRAD_004,home,41.252199,-95.994082
4,RWRAD_005,home,41.235652,-95.965909


In [190]:
# Filter the 'home_gps_info' DataFrame to keep only those rows where 'subj' is not in 'final_output_df'
filtered_home_gps_info = home_gps_info[~home_gps_info['subj'].isin(home_df_first_entry['subj'])]

In [192]:
len(filtered_home_gps_info)

46

In [194]:
filtered_home_gps_info.head()

,subj,type,gps_lat,gps_long
11,RWRAD_012,home,41.245465,-96.189452
13,RWRAD_014,home,41.299023,-96.113945
15,RWRAD_016,home,41.155644,-95.926620
20,RWRAD_021,home,41.482989,-95.223222
31,RWRAD_032,home,41.267487,-96.018009


In [200]:
# Save the filtered DataFrame to CSV
filtered_home_gps_info.to_csv('home_information_for_remaining_subjects.csv', index=False)

## Do the same process for the remaining csv files

In [205]:
import os
import pandas as pd
from geopy.distance import geodesic

# Initialize the final output dataframe
final_home_gps_info = pd.DataFrame(columns=['subj', 'drive', 'distance', 'label'])

# Path to the GPS data files
gps_files_path = r"C:\Users\aparnaj8\Box\Data (Matthew Rizzo)\DataAnalysis\ReferenceDatabases\BlackBox_GPSDrivingLocations\BlackBox_GPSDrivingLocations_1hz"

# Get all CSV files from the folder
gps_files = [file for file in os.listdir(gps_files_path) if file.endswith(".csv")]

# Iterate through all CSV files
for file in gps_files:
    # Load the current GPS file
    gps_df = pd.read_csv(os.path.join(gps_files_path, file))
    gps_df = gps_df[gps_df['time_cat'] == 'start']  # Filter for 'start' only

    # Print which file is being processed
    print(f"Processing file: {file}")

    # Iterate over each row in filtered_home_gps_info to match with GPS data
    for index, row in filtered_home_gps_info.iterrows():
        # Match the subject in the GPS data
        matching_gps = gps_df[gps_df['subj'] == row['subj']]

        if not matching_gps.empty:
            # Calculate the geodesic distance for each match if coordinates are valid
            home_lat = row['gps_lat']
            home_lon = row['gps_long']

            # Skip if the home coordinates are missing or invalid
            if pd.isna(home_lat) or pd.isna(home_lon):
                continue

            home_point = (home_lat, home_lon)
            
            # For each matching entry, calculate the distance and label
            for _, gps_row in matching_gps.iterrows():
                gps_lat = gps_row['gps_lat']
                gps_lon = gps_row['gps_long']
                
                # Skip if GPS coordinates are missing or invalid
                if pd.isna(gps_lat) or pd.isna(gps_lon):
                    continue

                gps_point = (gps_lat, gps_lon)

                # Calculate distance in feet
                distance = geodesic(home_point, gps_point).feet
                
                # Assign label based on distance
                label = 'home' if distance < 100 else 'not_home'
                
                # Prepare the new row as a DataFrame
                new_row = pd.DataFrame({
                    'subj': [row['subj']],
                    'drive': [gps_row['drive']],
                    'distance': [distance],
                    'label': [label]
                })
                
                # Concatenate the new row to the final dataframe
                final_home_gps_info = pd.concat([final_home_gps_info, new_row], ignore_index=True)



Processing file: RWRAD_001_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_002_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_003_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_004_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_005_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_006_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_007_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_008_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_009_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_010_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_011_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_012_BlackBox_GPSDrivingLocations_1hz.csv


C:\Users\aparnaj8\AppData\Local\Temp\ipykernel_21204\3680911501.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_home_gps_info = pd.concat([final_home_gps_info, new_row], ignore_index=True)


Processing file: RWRAD_013_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_014_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_015_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_016_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_017_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_018_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_019_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_020_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_021_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_022_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_023_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_024_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_025_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_026_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_027_BlackBox_GPSDrivingLocations_1hz.csv
Processing file: RWRAD_028_BlackBox_GPSD

In [219]:
final_home_gps_info.head()

,subj,drive,distance,label
0,RWRAD_012,1,30486.261448,not_home
1,RWRAD_012,3,1473.740612,not_home
2,RWRAD_012,5,27171.502752,not_home
3,RWRAD_012,6,24465.414520,not_home
4,RWRAD_012,7,930.783145,not_home


In [221]:
# Save the filtered DataFrame to CSV
final_home_gps_info.to_csv('RWRAD_start_trip_home_info_for_remaining_subjects.csv', index=False)

## Read the merged file

In [295]:
# Define the file path
file_path = '/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Trip_variable/RWRAD_start_trip_info_allsubjects.csv'

# Read the CSV file into a DataFrame
all_df = pd.read_csv(file_path)

In [297]:
all_df .head()

,subj,drive,distance,label
0,RWRAD_001,2,5867.960252,not_home
1,RWRAD_001,3,14367.450770,not_home
2,RWRAD_001,4,13534.904550,not_home
3,RWRAD_001,5,17.808028,home
4,RWRAD_001,6,11.872215,home


In [299]:
# Get the unique 'subj' values from all_df
unique_subj = all_df['subj'].unique()


# Optional: You can also get the count of unique subjects
unique_subj_count = len(unique_subj)
print(f"Number of unique subjects: {unique_subj_count}")

Number of unique subjects: 155


## Filter only those trips which started from home

In [301]:
# Filter rows where label is 'home'
home_df = all_df[all_df['label'] == 'home']

# Optional: Get the count of rows where label is 'home'
home_count = home_df.shape[0]
print(f"Number of rows with label 'home': {home_count}")


Number of rows with label 'home': 4200


In [303]:
# Get the unique 'subj' values from all_df
unique_subj = home_df['subj'].unique()

# Optional: You can also get the count of unique subjects
unique_subj_count = len(unique_subj)
print(f"Number of unique subjects: {unique_subj_count}")

Number of unique subjects: 146


In [305]:
home_df.head()

,subj,drive,distance,label
3,RWRAD_001,5,17.808028,home
4,RWRAD_001,6,11.872215,home
9,RWRAD_001,13,83.203012,home
13,RWRAD_001,18,66.713582,home
17,RWRAD_001,23,46.298697,home


In [246]:
# Save the filtered DataFrame to CSV
home_df.to_csv('RWRAD_start_trip_home_only_for_all_subjects.csv', index=False)

## Find trips which are <15 or 25 miles from home

In [414]:
# Define the file path
file_path = '/Users/aparnaj8/Box/Data (Matthew Rizzo)\DataAnalysis\FormattingQAChecks\BlackBox_FormattingQAChecks\BlackBox_FormatQA_Data\DataSummaries_1hz/BlackBox_1hz_byDriveSummary.csv'

# Read the CSV file into a DataFrame
drive_summary_df= pd.read_csv(file_path)

In [422]:
drive_summary_df['time_cat'].unique()

array(['morning', 'afternoon', 'evening', nan, 'night'], dtype=object)

### Save only required variables


In [150]:
# Keep only the specified columns
filtered_drive_summary_df = drive_summary_df[['subj', 'drive', 'time_start_utc', 'time_end_utc', 'time_start_cst', 'time_end_cst', 'time_weekday', 'duration_minutes', 'distance_miles', 'speed_mph_mean']]


In [152]:
filtered_drive_summary_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364


In [169]:
# Merge the two DataFrames on 'subj' and 'drive'
merged_df = pd.merge(filtered_drive_summary_df, home_df[['subj', 'drive', 'distance', 'label']], on=['subj', 'drive'], how='left')

# Rename the 'distance' column to 'calc_distance'
merged_df.rename(columns={'distance': 'home_distance'}, inplace=True)



In [171]:
merged_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,home_distance,label
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,NaN,NaN
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,NaN,NaN
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,NaN,NaN
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,NaN,NaN
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,17.808028,home


In [173]:
len(merged_df)

46267

# Add 15 miles and 25 miles column

In [175]:
import numpy as np

# Create a new column '15_miles_from_home' where the distance is less than 15 miles and label is 'home'
merged_df['15_miles_from_home'] = merged_df.apply(
    lambda row: 'yes' if row['label'] == 'home' and row['distance_miles'] < 15 else ('no' if row['label'] == 'home' else np.nan), axis=1)

# Create a new column '25_miles_from_home' where the distance is less than 25 miles and label is 'home'
merged_df['25_miles_from_home'] = merged_df.apply(
    lambda row: 'yes' if row['label'] == 'home' and row['distance_miles'] < 25 else ('no' if row['label'] == 'home' else np.nan), axis=1)



In [187]:
# List of columns to convert to datetime
columns_to_convert = ['time_start_utc', 'time_end_utc', 'time_start_cst', 'time_end_cst']  # Replace with actual column names

# Convert specified columns to datetime format
for column in columns_to_convert:
    merged_df[column] = pd.to_datetime(merged_df[column], errors='coerce')



In [189]:
merged_df.dtypes

subj                          object
drive                          int64
time_start_utc        datetime64[ns]
time_end_utc          datetime64[ns]
time_start_cst        datetime64[ns]
time_end_cst          datetime64[ns]
time_weekday                  object
duration_minutes             float64
distance_miles               float64
speed_mph_mean               float64
home_distance                float64
label                         object
15_miles_from_home            object
25_miles_from_home            object
dtype: object

In [193]:
# Save the filtered DataFrame to CSV
merged_df.to_csv('RWRAD_15_25_miles_from_home.csv', index=False)

In [ ]:
# Count occurrences of 'yes', 'no', and NaN for the '15_miles_from_home' column
count_15_miles = merged_df['15_miles_from_home'].value_counts(dropna=False)

# Count occurrences of 'yes', 'no', and NaN for the '25_miles_from_home' column
count_25_miles = merged_df['25_miles_from_home'].value_counts(dropna=False)

# Display the counts
print("Count for '15_miles_from_home':")
print(count_15_miles)

print("\nCount for '25_miles_from_home':")
print(count_25_miles)


In [225]:
len(merged_df)

46267

## Attach Trip Chaining Information

In [ ]:
# Define the file path
file_path = '/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Trip_information/RWRAD_15_25_miles_from_home_week_info.csv'

# Read the CSV file into a DataFrame
trip_summary_df= pd.read_csv(file_path)

In [ ]:
trip_summary_df.head()

In [ ]:
# Define the file path
file_path = '/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Trip_variable/soura_start_end_rwrad.csv'

# Read the CSV file into a DataFrame
start_df= pd.read_csv(file_path)

In [ ]:
# Filter the dataframe based on the conditions
start_df_filtered = start_df[(start_df['time_cat'] == 'start')]

In [442]:
start_df_filtered.head()

,Unnamed: 0,subj,drive,time_cat,time_utc,gps_lat,gps_long,labels
0,3.0,RWRAD_001,2,start,2021-03-18 17:06:48,40.768795,-96.623795,unc
2,6.0,RWRAD_001,3,start,2021-03-18 17:36:45,40.815170,-96.629127,unc
4,9.0,RWRAD_001,4,start,2021-03-18 17:59:47,40.812519,-96.629463,unc
7,14.0,RWRAD_001,5,start,2021-03-18 20:36:08,40.779293,-96.607643,home
8,16.0,RWRAD_001,6,start,2021-03-18 22:30:18,40.779282,-96.607658,home


In [444]:
# Define the file path
file_path = '/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Souradeep_Trip_Info/RWRAD_all_label_wdrive.csv'

# Read the CSV file into a DataFrame
end_df= pd.read_csv(file_path)

In [446]:
# Get the unique 'subj' values from all_df
unique_subj = end_df['labels'].unique()
unique_subj

array(['home', 'work', 'errand', 'social', 'medical', 'none'],
      dtype=object)

In [448]:
end_df.head()

,Unnamed: 0,subj,drive,gps_lat,gps_long,time_utc,labels
0,10.0,RWRAD_001,4,40.779259,-96.607689,2021-03-18 18:21:48,home
1,13.0,RWRAD_001,5,40.779266,-96.607674,2021-03-18 21:23:42,home
2,20.0,RWRAD_001,7,40.779259,-96.607796,2021-03-18 23:58:10,home
3,25.0,RWRAD_001,9,40.779236,-96.607628,2021-03-19 14:30:10,home
4,31.0,RWRAD_001,12,40.779263,-96.607636,2021-03-19 15:30:53,home


In [450]:
trip_summary_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,home_distance,label,15_miles_from_home,25_miles_from_home,year,year_mapped,subj_type,age,week_number
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,NaN,NaN,NaN,NaN,2021,1,mci,79.0,11
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,NaN,NaN,NaN,NaN,2021,1,mci,79.0,11
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,NaN,NaN,NaN,NaN,2021,1,mci,79.0,11
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,NaN,NaN,NaN,NaN,2021,1,mci,79.0,11
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,17.808028,home,yes,yes,2021,1,mci,79.0,11


#### Attach start and end home information to the dataframe

In [453]:
# Add start_location and end_location columns with default value
trip_summary_df["start_location"] = None
trip_summary_df["end_location"] = None

# Create a mapping dictionary for end_df (subj, drive) -> label
start_location_mapping = start_df_filtered.set_index(["subj", "drive"])["labels"].to_dict()

# Update the end_location column using the mapping
trip_summary_df["start_location"] = trip_summary_df.set_index(["subj", "drive"]).index.map(start_location_mapping)

# Create a mapping dictionary for end_df (subj, drive) -> label
end_location_mapping = end_df.set_index(["subj", "drive"])["labels"].to_dict()

# Update the end_location column using the mapping
trip_summary_df["end_location"] = trip_summary_df.set_index(["subj", "drive"]).index.map(end_location_mapping)

# Convert None to a default value, e.g., "unknown" (if needed)
trip_summary_df["end_location"].fillna("None", inplace=True)

# Convert None to a default value, e.g., "unknown" (if needed)
trip_summary_df["start_location"].fillna("None", inplace=True)

In [455]:
trip_summary_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,label,15_miles_from_home,25_miles_from_home,year,year_mapped,subj_type,age,week_number,start_location,end_location
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,NaN,NaN,NaN,2021,1,mci,79.0,11,None,None
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,NaN,NaN,NaN,2021,1,mci,79.0,11,unc,social
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,NaN,NaN,NaN,2021,1,mci,79.0,11,unc,errand
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,NaN,NaN,NaN,2021,1,mci,79.0,11,unc,home
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,home,yes,yes,2021,1,mci,79.0,11,home,home


#### Count the drives which started at home but ended somewhere else

In [457]:
# Count where start_location is "home" and end_location is None
count_start_home_end_none = trip_summary_df[
    (trip_summary_df["start_location"] == "home") & 
    (trip_summary_df["end_location"]!= "home")
].drop_duplicates(subset=["subj", "drive"]).shape[0]

# Count where start_location is "home" and end_location is "home"
count_start_home_end_home = trip_summary_df[
    (trip_summary_df["start_location"] == "home") & 
    (trip_summary_df["end_location"] == "home")
].drop_duplicates(subset=["subj", "drive"]).shape[0]

# Print results
print(f"Count of (start_location = home, end_location = None): {count_start_home_end_none}")
print(f"Count of (start_location = home, end_location = home): {count_start_home_end_home}")


Count of (start_location = home, end_location = None): 3163
Count of (start_location = home, end_location = home): 1030


In [459]:
len(trip_summary_df)

46267

#### calculate time difference or stoppage between drives


In [462]:
import pandas as pd

# Ensure the columns are datetime objects
trip_summary_df["time_start_utc"] = pd.to_datetime(trip_summary_df["time_start_utc"])
trip_summary_df["time_end_utc"] = pd.to_datetime(trip_summary_df["time_end_utc"])

# Sort the DataFrame by subject and drive
trip_summary_df = trip_summary_df.sort_values(by=["subj", "drive"])

# Define a function to compute time differences within groups
def calculate_time_diff(group):
    group = group.sort_values(by="drive")  # Ensure drives are sorted
    group["time_diff_minutes"] = (
        group["time_start_utc"] - group["time_end_utc"].shift(1)
    ).dt.total_seconds() / 60
    return group

# Apply the function to each group and reset the index
trip_summary_df = trip_summary_df.groupby("subj", group_keys=False).apply(calculate_time_diff)



In [464]:
trip_summary_df.head()


,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,15_miles_from_home,25_miles_from_home,year,year_mapped,subj_type,age,week_number,start_location,end_location,time_diff_minutes
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,NaN,NaN,2021,1,mci,79.0,11,None,None,NaN
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,NaN,NaN,2021,1,mci,79.0,11,unc,social,4.350000
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,NaN,NaN,2021,1,mci,79.0,11,unc,errand,12.650000
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,NaN,NaN,2021,1,mci,79.0,11,unc,home,19.850000
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,yes,yes,2021,1,mci,79.0,11,home,home,134.333333


In [466]:
# Filter rows where both start_location and end_location are "home"
home_filter = (trip_summary_df["start_location"] == "home") & (trip_summary_df["end_location"] == "home")

# Get the row with the minimum value of distance_miles
min_distance_row = trip_summary_df.loc[home_filter, "distance_miles"].idxmin()

# Get the row with the maximum value of distance_miles
max_distance_row = trip_summary_df.loc[home_filter, "distance_miles"].idxmax()

# Get the row with the minimum value of time_diff_minutes
min_time_row = trip_summary_df.loc[home_filter, "time_diff_minutes"].idxmin()

# Get the row with the maximum value of time_diff_minutes
max_time_row = trip_summary_df.loc[home_filter, "time_diff_minutes"].idxmax()

# Print the subj and drive information for these rows
min_max_info = {
    "Min Distance Miles": trip_summary_df.loc[min_distance_row, ["subj", "drive", "distance_miles"]],
    "Max Distance Miles": trip_summary_df.loc[max_distance_row, ["subj", "drive", "distance_miles"]],
    "Min Time Diff": trip_summary_df.loc[min_time_row, ["subj", "drive", "time_diff_minutes"]],
    "Max Time Diff": trip_summary_df.loc[max_time_row, ["subj", "drive", "time_diff_minutes"]]
}

# Display the result
for key, value in min_max_info.items():
    print(f"{key}: {value}\n")


Min Distance Miles: subj              RWRAD_080
drive                   319
distance_miles          0.0
Name: 21827, dtype: object

Max Distance Miles: subj              RWRAD_019
drive                   181
distance_miles    82.630137
Name: 5426, dtype: object

Min Time Diff: subj                 RWRAD_131
drive                      131
time_diff_minutes     2.033333
Name: 36107, dtype: object

Max Time Diff: subj                    RWRAD_129
drive                          39
time_diff_minutes    29958.633333
Name: 35678, dtype: object



#### calculate the trip chain information

In [484]:
import pandas as pd

# Ensure the DataFrame is sorted for sequential processing
trip_summary_df = trip_summary_df.sort_values(by=["subj", "drive"])

# Initialize the `trip_chain` column
trip_summary_df['trip_chain'] = None

# Initialize global chain number
global_chain_number = 0

# Iterate over each unique subject
for subj in trip_summary_df['subj'].unique():
    # Filter trips for the current subject
    subj_df = trip_summary_df[trip_summary_df['subj'] == subj]
    
    # Variables to track chain state
    in_chain = False  # Flag for active chain
    
    for idx, row in subj_df.iterrows():
        # Case 1: Standalone trips starting and ending at home
        if row['start_location'] == 'home' and row['end_location'] == 'home':
            global_chain_number += 1  # Increment global chain number
            trip_summary_df.at[idx, 'trip_chain'] = global_chain_number  # Assign chain number
            in_chain = False  # Reset chain flag

        # Case 2: Trip starts at home but does not end at home
        elif row['start_location'] == 'home' and row['end_location'] != 'home':
            if not in_chain:  # Start a new chain if not already in one
                global_chain_number += 1
            trip_summary_df.at[idx, 'trip_chain'] = global_chain_number
            in_chain = True  # Mark that we're in a chain

        # Case 3: Trip does not start at home but ends at home
        elif row['start_location'] != 'home' and row['end_location'] == 'home':
            if in_chain:  # If already in a chain, continue it
                trip_summary_df.at[idx, 'trip_chain'] = global_chain_number
                in_chain = False  # End the chain

        # Case 4: Other trips
        else:
            if in_chain:  # Continue the current chain
                trip_summary_df.at[idx, 'trip_chain'] = global_chain_number


In [490]:
trip_summary_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,25_miles_from_home,year,year_mapped,subj_type,age,week_number,start_location,end_location,time_diff_minutes,trip_chain
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,NaN,2021,1,mci,79.0,11,None,None,NaN,None
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,NaN,2021,1,mci,79.0,11,unc,social,4.350000,None
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,NaN,2021,1,mci,79.0,11,unc,errand,12.650000,None
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,NaN,2021,1,mci,79.0,11,unc,home,19.850000,None
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,yes,2021,1,mci,79.0,11,home,home,134.333333,1


In [488]:
# Save the filtered DataFrame to CSV
#trip_summary_df.to_csv('trip_chain.csv', index=False)

### Take into account dwell time =30min to terminate the trip

In [39]:
# Define the file path
file_path = '/Users/aparn/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Trip_chain/trip_chain_v2.csv'

# Read the CSV file into a DataFrame
trip_summary_df= pd.read_csv(file_path)

In [51]:
trip_summary_df.head(10)

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,25_miles_from_home,year,year_mapped,subj_type,age,week_number,start_location,end_location,time_diff_minutes,trip_chain
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,NaN,2021,1,mci,79.0,11,NaN,NaN,NaN,NaN
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,NaN,2021,1,mci,79.0,11,unc,social,4.350000,NaN
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,NaN,2021,1,mci,79.0,11,unc,errand,12.650000,NaN
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,NaN,2021,1,mci,79.0,11,unc,home,19.850000,NaN
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,yes,2021,1,mci,79.0,11,home,home,134.333333,1.0
5,RWRAD_001,6,2021-03-18 22:30:18,2021-03-18 22:39:59,2021-03-18 17:30:18,2021-03-18 17:39:59,thursday,9.683333,3.823141,23.688999,...,yes,2021,1,mci,79.0,11,home,social,66.600000,2.0
6,RWRAD_001,7,2021-03-18 23:35:49,2021-03-18 23:58:10,2021-03-18 18:35:49,2021-03-18 18:58:10,thursday,22.350000,8.432317,22.637092,...,NaN,2021,1,mci,79.0,11,unc,home,55.833333,2.0
7,RWRAD_001,8,2021-03-19 12:50:20,2021-03-19 13:09:19,2021-03-19 07:50:20,2021-03-19 08:09:19,friday,18.983333,3.525976,11.213353,...,NaN,2021,1,mci,79.0,11,NaN,NaN,772.166667,NaN
8,RWRAD_001,9,2021-03-19 14:04:19,2021-03-19 14:30:10,2021-03-19 09:04:19,2021-03-19 09:30:10,friday,25.850000,5.472138,13.428561,...,NaN,2021,1,mci,79.0,11,unc,home,55.000000,NaN
9,RWRAD_001,10,2021-03-19 14:38:41,2021-03-19 14:51:34,2021-03-19 09:38:41,2021-03-19 09:51:34,friday,12.883333,5.069708,25.597404,...,yes,2021,1,mci,79.0,11,home,medical,8.516667,3.0


In [41]:
# Extract the specified columns into a new DataFrame
trip_chain_check = trip_summary_df[['subj', 'drive', 'start_location', 'end_location', 'time_diff_minutes', 'trip_chain']]

In [42]:
import pandas as pd
import numpy as np

# Create a copy of the DataFrame
updated_df = trip_summary_df.copy()

# Identify 'trip_chain' values that appear more than once
trip_chain_counts = updated_df['trip_chain'].value_counts()
trip_chains_to_modify = trip_chain_counts[trip_chain_counts > 1].index

# Iterate through the rows for trip chains with duplicates
for trip_chain in trip_chains_to_modify:
    # Get indices of rows corresponding to the current trip_chain
    indices = updated_df[updated_df['trip_chain'] == trip_chain].index
    # Set the 'time_diff_minutes' value to NaN for the first occurrence only
    updated_df.loc[indices[0], 'time_diff_minutes'] = np.nan

In [52]:
updated_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,year,year_mapped,subj_type,age,week_number,start_location,end_location,time_diff_minutes,trip_chain,trip_chain_check
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,2021,1,mci,79.0,11,NaN,NaN,NaN,NaN,NaN
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,2021,1,mci,79.0,11,unc,social,4.350000,NaN,NaN
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,2021,1,mci,79.0,11,unc,errand,12.650000,NaN,NaN
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,2021,1,mci,79.0,11,unc,home,19.850000,NaN,NaN
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,2021,1,mci,79.0,11,home,home,134.333333,1.0,1.0


In [45]:
# Create a new column 'trip_chain_check' as a copy of 'trip_chain'
updated_df['trip_chain_check'] = updated_df['trip_chain']

# Get the unique values of 'trip_chain' that appear more than once
trip_chain_counts = updated_df['trip_chain'].value_counts()
duplicate_trip_chains = trip_chain_counts[trip_chain_counts > 1].index

# Iterate over duplicate trip chains and process 'time_diff_minutes'
for trip_chain in duplicate_trip_chains:
    # Filter rows for the current trip chain
    rows = updated_df[updated_df['trip_chain'] == trip_chain]
    
    # Check if any 'time_diff_minutes' values are greater than 30 (ignore NaNs)
    if rows['time_diff_minutes'].dropna().gt(30).any():
        # Set 'trip_chain_check' for these rows to NaN
        updated_df.loc[updated_df['trip_chain'] == trip_chain, 'trip_chain_check'] = np.nan



In [53]:
updated_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,year,year_mapped,subj_type,age,week_number,start_location,end_location,time_diff_minutes,trip_chain,trip_chain_check
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,2021,1,mci,79.0,11,NaN,NaN,NaN,NaN,NaN
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,2021,1,mci,79.0,11,unc,social,4.350000,NaN,NaN
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,2021,1,mci,79.0,11,unc,errand,12.650000,NaN,NaN
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,2021,1,mci,79.0,11,unc,home,19.850000,NaN,NaN
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,2021,1,mci,79.0,11,home,home,134.333333,1.0,1.0


In [48]:
updated_df['trip_chain_check'].nunique()

1811

In [54]:
# Ensure the 'trip_chain_check' column exists in updated_df
if 'trip_chain_check' in updated_df.columns:
    # Merge 'trip_chain_check' from updated_df into trip_summary_df based on keys
    trip_summary_df = trip_summary_df.merge(
        updated_df[['subj', 'drive', 'trip_chain', 'trip_chain_check']],
        on=['subj', 'drive', 'trip_chain'],
        how='left'
    )


In [55]:
trip_summary_df['trip_chain_check'].nunique()

1811

In [56]:
# Save the filtered DataFrame to CSV
#trip_summary_df.to_csv('trip_chain_dwell_time.csv', index=False)

## correct 15miles 25miles from home variable (there are some more drives which start from home)

In [27]:
# Update 'label' column wherever 'start_location' contains 'home'
trip_summary_df.loc[trip_summary_df['start_location'].str.contains('home', case=False, na=False), 'label'] = 'home'


In [28]:
import numpy as np

# Create a new column '15_miles_from_home' where the distance is less than 15 miles and label is 'home'
trip_summary_df['15_miles_from_home'] = trip_summary_df.apply(
    lambda row: 'yes' if row['label'] == 'home' and row['distance_miles'] < 15 else ('no' if row['label'] == 'home' else np.nan), axis=1)

# Create a new column '25_miles_from_home' where the distance is less than 25 miles and label is 'home'
trip_summary_df['25_miles_from_home'] = trip_summary_df.apply(
    lambda row: 'yes' if row['label'] == 'home' and row['distance_miles'] < 25 else ('no' if row['label'] == 'home' else np.nan), axis=1)



In [29]:
# Save the filtered DataFrame to CSV
trip_summary_df.to_csv('trip_chain_v2.csv', index=False)